In [3]:
!pip install transformers datasets accelerate sentencepiece faiss-cpu
!pip install einops
!pip install handystuff

In [4]:
"""
Colab launcher for Retomaton/Base/Fine-tuned LLM evaluation.
Replaces the bash script entirely.

This script:
• Mounts Google Drive
• Sets all interpolation, dstore, and evaluation parameters
• Calls your existing Retomaton generation script (5_generations_perplexity_debug.py)
• Streams logs to console + persistent log file
"""

import os
import subprocess
from datetime import datetime

# ============================================================
# 1. MOUNT GOOGLE DRIVE
# ============================================================
print("[INFO] Mounting Google Drive...")
from google.colab import drive
drive.mount('/content/drive')


[INFO] Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# ============================================================
# 2. CONFIGURATION (EDIT THESE PATHS TO MATCH YOUR SETUP)
# ============================================================

# Base directory where everything lives
BASE_DIR = "/content/drive/MyDrive/faiss_mts_olmo/"

# Path to generation script (your Python version)
GEN_SCRIPT = "/content/drive/MyDrive/faiss_mts_olmo/core_scripts/5_generations_perplexity_debug3.py"

# Repo path containing knnlm.py + retomaton.py
REPO_PATH = "/content/drive/MyDrive/faiss_mts_olmo/core_scripts/"

# Models
MODEL_NAME = "allenai/OLMo-2-0425-1B-Instruct"
FINETUNED_MODEL = "allenai/OLMo-2-0425-1B-Instruct" #Replace with the actual finetuned model - this is just for demonstration.

# Dataset (JSONL)
JSONL = BASE_DIR + 'execution_scripts/mock_datasets_radiology_jsonl/' + "mimic_inspired_test_context_impression-finetune.jsonl"

# Datastore (already built)
DSTORE_DIR = "/content/drive/MyDrive/faiss_mts_olmo/execution_scripts/datastores/allenai_OLMo-2-0425-1B-Instruct"
DSTORE_SIZE = 237510
DIM = 2048

# Retrieval hyperparameters
K = 1024
LAMBDA = 0.5
KNN_TEMP = 1.0
MIN_KNN = 500
MAX_KNN = 2048

# Generation hyperparameters
TEMP = 0.0          # deterministic
TOPP = 1.0
LIMIT = 60          # number of evaluation items

In [22]:
JSONL

'/content/drive/MyDrive/faiss_mts_olmo/Attempts_27Dec25/mock_datasets_radiology_jsonl/mimic_inspired_test_context_impression-finetune.jsonl'

In [23]:
# ============================================================
# 3. OUTPUT DIRECTORY
# ============================================================

OUTPUT_DIR = os.path.join(
    BASE_DIR,
    f"generations_k{K}_lambda{LAMBDA}_min{MIN_KNN}_max{MAX_KNN}_T{TEMP}_p{TOPP}"
)

os.makedirs(OUTPUT_DIR, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
LOG_FILE = os.path.join(OUTPUT_DIR, f"run_{timestamp}.log")

print(f"[INFO] Output directory: {OUTPUT_DIR}")
print(f"[INFO] Log file: {LOG_FILE}")


[INFO] Output directory: /content/drive/MyDrive/faiss_mts_olmo/generations_k1024_lambda0.5_min500_max2048_T0.0_p1.0
[INFO] Log file: /content/drive/MyDrive/faiss_mts_olmo/generations_k1024_lambda0.5_min500_max2048_T0.0_p1.0/run_20251227_091852.log


In [24]:
# ============================================================
# 4. BUILD COMMAND (exact equivalent of your bash)
# ============================================================

cmd = [
    "python", "-u", GEN_SCRIPT,
    "--repo_path", REPO_PATH,
    "--model_name", MODEL_NAME,
    "--finetuned_dir", FINETUNED_MODEL,
    "--jsonl", JSONL,
    "--output_dir", OUTPUT_DIR,
    "--dstore_dir", DSTORE_DIR,
    "--dstore_size", str(DSTORE_SIZE),
    "--dimension", str(DIM),
    "--k", str(K),
    "--lambda_val", str(LAMBDA),
    "--knn_temp", str(KNN_TEMP),
    "--min_knn", str(MIN_KNN),
    "--max_knn", str(MAX_KNN),
    "--temperature", str(TEMP),
    "--top_p", str(TOPP),
    "--limit", str(LIMIT),
]

print("\n[INFO] Running command:")
print(" ".join(cmd))
print()


[INFO] Running command:
python -u /content/drive/MyDrive/faiss_mts_olmo/core_scripts/5_generations_perplexity_debug3.py --repo_path /content/drive/MyDrive/faiss_mts_olmo/core_scripts/ --model_name allenai/OLMo-2-0425-1B-Instruct --finetuned_dir allenai/OLMo-2-0425-1B-Instruct --jsonl /content/drive/MyDrive/faiss_mts_olmo/Attempts_27Dec25/mock_datasets_radiology_jsonl/mimic_inspired_test_context_impression-finetune.jsonl --output_dir /content/drive/MyDrive/faiss_mts_olmo/generations_k1024_lambda0.5_min500_max2048_T0.0_p1.0 --dstore_dir /content/drive/MyDrive/faiss_mts_olmo/Attempts_27Dec25/datastores/allenai_OLMo-2-0425-1B-Instruct --dstore_size 237510 --dimension 2048 --k 1024 --lambda_val 0.5 --knn_temp 1.0 --min_knn 500 --max_knn 2048 --temperature 0.0 --top_p 1.0 --limit 60



In [25]:
import faiss

In [26]:
# ============================================================
# 5. RUN + LIVE LOGGING
# ============================================================

with open(LOG_FILE, "w") as lf:
    process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        universal_newlines=True,
        bufsize=1
    )

    print("[INFO] Generation started. Streaming logs:\n")
    for line in process.stdout:
        print(line, end="")
        lf.write(line)

process.wait()

print("\n[INFO] Completed.")
print(f"[INFO] Results saved in: {OUTPUT_DIR}")
print(f"[INFO] Full log at: {LOG_FILE}")

[INFO] Generation started. Streaming logs:

Using device: cuda
`torch_dtype` is deprecated! Use `dtype` instead!
2025-12-27 09:19:03.794474: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-27 09:19:03.815765: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766827143.837296    5349 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766827143.842745    5349 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766827143.856952 